In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf 


from tensorflow.keras import Sequential, utils
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

train_dir = os.path.join(dataset_dir, 'train')

print("train_dir: ", os.listdir(train_dir))

remove_dir = os.path.join(train_dir, 'unsup')

shutil.rmtree(remove_dir)


train_dir:  ['labeledBow.feat', 'neg', 'unsupBow.feat', 'urls_pos.txt', 'pos', 'urls_neg.txt', 'urls_unsup.txt', 'unsup']


In [3]:
batch_size = 1024
seed = 123

train_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)

val_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='validation', seed=seed)

for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print(label_batch[i].numpy(), text_batch.numpy()[i])


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the

In [4]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

embedding_layer = tf.keras.layers.Embedding(1000, 5)


result = embedding_layer(tf.constant([1,2,3]))
print(result.numpy())

[[-3.5796724e-02  8.6009502e-05  2.4448182e-02 -4.2200781e-02
   2.8508518e-02]
 [-3.9833881e-02 -1.7407764e-02  3.8590994e-02  3.8167704e-02
   2.6471380e-02]
 [-1.1200439e-02 -3.3491910e-02  1.6295541e-02  2.8785978e-02
   7.2355494e-03]]


In [5]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')


In [6]:
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)


In [7]:

embedding_dim = 16
model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name='embedding'),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])


In [8]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')


In [9]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [10]:
model.fit(train_ds, validation_data=val_ds, epochs=15, callbacks=[tensorboard_callback])


Epoch 1/15
20/20 [==============================] - 5s 197ms/step - loss: 0.6924 - accuracy: 0.5037 - val_loss: 0.6888 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 100ms/step - loss: 0.6865 - accuracy: 0.5037 - val_loss: 0.6802 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 92ms/step - loss: 0.6766 - accuracy: 0.5037 - val_loss: 0.6676 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 93ms/step - loss: 0.6619 - accuracy: 0.5037 - val_loss: 0.6502 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 93ms/step - loss: 0.6419 - accuracy: 0.5038 - val_loss: 0.6282 - val_accuracy: 0.4922
Epoch 6/15
20/20 [==============================] - 2s 91ms/step - loss: 0.6165 - accuracy: 0.5255 - val_loss: 0.6024 - val_accuracy: 0.5596
Epoch 7/15
20/20 [==============================] - 2s 95ms/step - loss: 0.5867 - accuracy: 0.5991 - val_loss: 0.5743 - val_accuracy: 0.6236
Epoch 8/15


In [11]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
%load_ext tensorboard
%tensorboard --logdir logs

In [13]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [14]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()